In [118]:
import numpy as np
import os
import pandas as pd
from PIL import Image
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [126]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [109]:
class ImagePreprocessingPipeline(object):
    def __init__(self,directory):
        self.directory = directory
        self.image_array = None
        self.scaler = StandardScaler()
        for filename in os.listdir(directory):
            f = os.path.join(directory, filename)
            # checking if it is a file
            if os.path.isfile(f):
                img = Image.open(f)
                #(n,h,w,1) is the preferred array shape for using a Keras CNN, where...
                # n = number of observations
                # h = image height
                # w = image width
                img_array = np.asarray(img).reshape(1,-1)
                if type(self.image_array) == type(img_array) and self.image_array.shape[1]==img_array.shape[1]:
                    self.image_array = np.concatenate((self.image_array,img_array))
                    if self.image_array.shape[0]%5000==0:
                        print("Progress!", self.image_array.shape)
                else:
                    self.image_array = img_array
                    print('First image converted to numpy array. This message should only occur once')
        self.image_array = self.scaler.fit_transform(self.image_array).reshape((self.image_array.shape[0],48,48,1))
        

In [110]:
training_images = ImagePreprocessingPipeline(directory="C:/Users/Owner/Documents/GradSchool/Current/MachineLearning/Projects/mdm/mdm_images/galaxy_postage/training")

First image converted to numpy array. This message should only occur once
Progress! (5000, 2304)
Progress! (10000, 2304)
Progress! (15000, 2304)
Progress! (20000, 2304)
Progress! (25000, 2304)
Progress! (30000, 2304)
Progress! (35000, 2304)
Progress! (40000, 2304)


In [111]:
print(training_images.image_array.shape)

(40000, 48, 48, 1)


In [116]:
training_star_images = ImagePreprocessingPipeline(directory="C:/Users/Owner/Documents/GradSchool/Current/MachineLearning/Projects/mdm/mdm_images/star_postage/training")

First image converted to numpy array. This message should only occur once
Progress! (5000, 2304)
Progress! (10000, 2304)
Progress! (15000, 2304)
Progress! (20000, 2304)
Progress! (25000, 2304)
Progress! (30000, 2304)
Progress! (35000, 2304)
Progress! (40000, 2304)


In [143]:
training_y = np.asarray(pd.read_csv("C:/Users/Owner/Documents/GradSchool/Current/MachineLearning/Projects/mdm/mdm_training_solution_sorted.csv")).astype('float32')[:,1:]
print(training_y.shape)
print(type(training_y))
print(training_images.image_array.shape)
print(type(training_images.image_array))


(40000, 2)
<class 'numpy.ndarray'>
(40000, 48, 48, 1)
<class 'numpy.ndarray'>


In [152]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(48,48,1)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(Dense(2, activation='softmax'))

model.compile(
    loss=MeanSquaredError(),
    optimizer=Adam(),
    metrics=['accuracy'])

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 46, 46, 32)        320       
_________________________________________________________________
leaky_re_lu_36 (LeakyReLU)   (None, 46, 46, 32)        0         
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 21, 21, 64)        18496     
_________________________________________________________________
leaky_re_lu_37 (LeakyReLU)   (None, 21, 21, 64)        0         
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 8, 8, 128)        

In [153]:
model_trained = model.fit(
    training_images.image_array,
    training_y,
    epochs=10)

Epoch 1/10
1250/1250 [==============================] - 13s 5ms/step - loss: 0.2727 - accuracy: 0.4977
Epoch 2/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.2726 - accuracy: 0.4988
Epoch 3/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.2726 - accuracy: 0.4989
Epoch 4/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.2726 - accuracy: 0.4999
Epoch 5/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.2726 - accuracy: 0.5028
Epoch 6/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.2726 - accuracy: 0.5034
Epoch 7/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.2726 - accuracy: 0.4994
Epoch 8/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.2726 - accuracy: 0.5009
Epoch 9/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.2726 - accuracy: 0.4987
Epoch 10/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.27